Box_analysis.py --- Last updated 09/22/22
Contact: James Sayre, sayrejay@gmail.com

Written to take sample boxes and the downloaded satellite imagery, subset the satellite imagery to our boxes of interest, and then run the analysis/histogram computation within each box.

Inputs:
- Within the each subfolder "Remote Sensing/Planet/STATECODE/MUNCODE/CYCLE_NAME_YEAR/", looks for a csv called mun_boxes_CYCLE_NAME_YEAR.csv (created by planet_api_querying.py) which details, for the relevant boxes of a given municipality, the correspondence between each box (the unique code for each box in this file is bid) and the satellite image it is contained in for that crop cycle and year (found within image_id). Note that some boxes may have multiple satellite images they are contained in, in which case the relevant overlapping areas are averaged to produce an average for the box.

- "data/muncodes/shp/MUNICIPIOS.shp" -- INEGI provided shapefile of municipality shapes, municipality codes + names

- Satellite imagery stored as GeoTIFFs, located within the folder "Remote Sensing/Planet/STATECODE/Raw/analytic_sr/" (older imagery has been stored under "analytic/", which doesn't have
bottom of atmosphere corrections necessary for agricultural change analysis)

Outputs:

- Up to you, currently outputs histograms by ADC

In [1]:
from box_analysis_utilities import *
muncode = 2001#int(sys.argv[1])
print("Muncode is", muncode)

### Inputs
mun_csv = os.path.join(base_dir,"data","muncodes","shp","MUNICIPIOS.csv")


/home/j/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.7.1-CAPI-1.11.1 0) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


Muncode is 2001


In [2]:
### Programs
def read_in_AOI_box_info(crop_cycle,year,municipality_id, asset_type='analytic_sr'):
    ### read_in_AOI_box_info -- In each folder "Remote Sensing/Planet/STATECODE/MUNCODE/Input/CropCycle_Year/",
    ### there is a csv file starting with "mun_boxes", that for each season and municipality, details the boxes
    ### that are areas of interest (AOIs) for that season and municipality, as well as listing the associated satellite 
    ### imagery available for each AOI. This program reads in both of those csvs (one corresponding to the start
    ### of the season, the other corresponding to the end), and subsets down to the AOIs for which we have satellite
    ## imagery available. Then returns those boxes as pd dataframes.
    chosen_state = str(municipality_id).replace(str(municipality_id)[-3:],"")
   
    ### Queries the relevant folder to see which satellite images have actually been downloaded
    img_fls_dled_dir = os.path.join(image_dir,chosen_state)
    input_fl_dir_srt = os.path.join(remote_sen_input_dir,chosen_state,str(municipality_id),crop_cycle+'_start_'+year)
    input_fl_dir_end = os.path.join(remote_sen_input_dir,chosen_state,str(municipality_id),crop_cycle+'_end_'+year)
    
    img_fls_dled = []
    for r,d,f in os.walk(img_fls_dled_dir):
        for file in f:
            if '_sr.tif' in file:
                img_fls_dled.append(file) 
    
    img_fls_dled = [img_fl.split('_sr.tif')[0].split('_a.tif')[0].split('.tif')[0] for img_fl in img_fls_dled]
    
    sz_start_df = os.path.join(input_fl_dir_srt,'mun_boxes_'+crop_cycle+'_start_'+year+'.csv')
    sz_end_df = os.path.join(input_fl_dir_end,'mun_boxes_'+crop_cycle+'_end_'+year+'.csv')    
    
    if os.path.isfile(sz_start_df):
        if os.path.isfile(sz_end_df):
            cycle_start_boxes_df = pd.read_csv(sz_start_df)
            cycle_end_boxes_df = pd.read_csv(sz_end_df)
            
            ### Drops AOIs with no satellite imagery available from Planet
            cycle_start_boxes_df = cycle_start_boxes_df[~cycle_start_boxes_df['image_id'].isnull()]
            cycle_end_boxes_df = cycle_end_boxes_df[~cycle_end_boxes_df['image_id'].isnull()]
            ### Drops AOIs that we haven't downloaded imagery for
            cycle_start_boxes_df['image_id'] = cycle_start_boxes_df['image_id'].apply(lambda x: [x for x in x.split(', ') if x in img_fls_dled])
            cycle_end_boxes_df['image_id'] = cycle_end_boxes_df['image_id'].apply(lambda x: [x for x in x.split(', ') if x in img_fls_dled])
            cycle_start_boxes_df = cycle_start_boxes_df[cycle_start_boxes_df['image_id'].apply(len) > 0]
            cycle_end_boxes_df = cycle_end_boxes_df[cycle_end_boxes_df['image_id'].apply(len) > 0]
        else: cycle_start_boxes_df, cycle_end_boxes_df = pd.DataFrame(), pd.DataFrame()
    else: cycle_start_boxes_df, cycle_end_boxes_df = pd.DataFrame(), pd.DataFrame()
        
    return cycle_start_boxes_df, cycle_end_boxes_df
    

def box_analysis(input_list, muncode, mun_name,
                 base_dir=base_dir,remote_sen_input_dir=remote_sen_input_dir, data_dir=data_dir,
                 remote_sen_output_dir=remote_sen_output_dir, image_dir=image_dir, 
                 eng_or_esp='eng', grid_width=1.5,
                 red_i = 2, nir_i = 3, ### Read docs for info on band positions: https://developers.planet.com/docs/data/reorthotile/
                 bright_scale = 5.0, ### Brightness scale for plotting output, 5 for SR products, 1 for analytic
                 asset = 'analytic_sr',
                  #### Whether or not you wish to plot and save the output of the workflow
                 should_i_save_png = False, should_i_save_bbox = False, should_i_plot = False):
    import geopandas as gpd
    import regionmask
    import shapely
    from shapely.geometry import Polygon
    from shapely.geometry import Point
    from shapely.ops import cascaded_union
    import xarray as xr
    
    stcode = str(muncode).replace(str(muncode)[-3:],"")
    if len(stcode) == 1:
        stcode = '0'+stcode
    fullmuncode     = stcode+str(muncode)[-3:]
    path_to_adc_shp = os.path.join(data_dir, "ADC", stcode, fullmuncode+'.shp')
    adc_df   = gpd.read_file(path_to_adc_shp)
    adc16_df = adc_df[~adc_df['adc16id'].isna()].drop('adcid',1).reset_index(drop=True)
    adc_df   = adc_df[~adc_df['adcid'].isna()].drop('adc16id',1).reset_index(drop=True)
    i_to_ca07_adc_dt = dict(zip(list(adc_df.index),list(adc_df['adcid'])))
    i_to_amca_adc_dt = dict(zip(list(adc16_df.index),list(adc16_df['adc16id'])))

    ### These will be used to save the information across boxes
    list_of_img_hist_adc07 = []     # this contains a list of all fallow da img histograms within mun
    list_of_img_hist_adc16 = []     # this contains a list of all crop da img histograms within mun
    
    for box in input_list:
        bid, xbox, ybox, image_ids_start, image_ids_end, ciclo, yr = box
#         print("Now processing box id:", bid)

        ### Create AOI box to subset imagery down to
        subset_poly_df, ytop, ybottom, xleft, xright = create_box_geodf(xbox,ybox,muncode)

        ### Read in, subset, and process imagery
        fallow_date = return_date_id(image_ids_start)
        fallow_da, fallow_crs = clean_up_downloaded_tif(image_ids=image_ids_start,
                                                        chosenmun=muncode, ciclo=ciclo+"_start_"+yr,
                                                        subset_df=subset_poly_df, asset_type=asset)
        crop_date = return_date_id(image_ids_end)
        crop_da, crop_crs = clean_up_downloaded_tif(image_ids=image_ids_end,
                                                    chosenmun=muncode, ciclo=ciclo+"_end_"+yr,
                                                    subset_df=subset_poly_df, asset_type=asset)
        
        

        ### Check if the images have matching projections
        no_crs_issue, crs_img = check_geo_proj_matching(fallow_crs,crop_crs)
        if no_crs_issue:
            num_bands_orig, xdim_cropda, ydim_cropda = crop_da.shape

            ### Make sure null parts match between images
            crop_da = crop_da.where(fallow_da.notnull())
            fallow_da = fallow_da.where(crop_da.notnull())
            if 'x' in crop_da.dims:
                if crop_da.isnull().all(dim=['x','y','band']).item():
                    crop_da_not_null = False
                else:
                    crop_da_not_null = True
            else:
                crop_da_not_null = False
            
            if 'x' in fallow_da.dims:
                if fallow_da.isnull().all(dim=['x','y','band']).item():
                    fallow_da_not_null = False
                else:
                    fallow_da_not_null = True
            else:
                fallow_da_not_null = False
                  
            if crop_da_not_null and fallow_da_not_null:
                if adc16_df.crs != fallow_crs:
                    adc16_df = adc16_df.to_crs(fallow_crs[0])
                if adc_df.crs != fallow_crs:
                    adc_df = adc_df.to_crs(fallow_crs[0])

                fallow_da = return_mask(fallow_da,adc_df,muncode=None,geo_col='adcid',mask_col='adc07')
                fallow_da = return_mask(fallow_da,adc16_df,muncode=None,geo_col='adc16id',mask_col='adc16')
                crop_da   = return_mask(crop_da,adc_df,muncode=None,geo_col='adcid',mask_col='adc07')
                crop_da   = return_mask(crop_da,adc16_df,muncode=None,geo_col='adc16id',mask_col='adc16')

                adc07hists, adc16hists = compute_histograms(crop_da,fallow_da,adc07dict=i_to_ca07_adc_dt,
                                                           adc16dict=i_to_amca_adc_dt,ciclo_yr=ciclo+yr)

                list_of_img_hist_adc07.extend(adc07hists)
                list_of_img_hist_adc16.extend(adc16hists)
                
    # Post-processing for image histograms for this bids list
    # add them up if they have the same ADC, ciclo and year
    total_img_hist_adc07 = total_img_hist(list_of_img_hist_adc07)
    total_img_hist_adc16 = total_img_hist(list_of_img_hist_adc16)

    return total_img_hist_adc07, total_img_hist_adc16

In [4]:
### Parameters
### A list of all the crop cycles and years
ciclos = [('winter','21'),('winter','20'),('winter','19'),('winter','18'),('winter','17'), 
          ('spring','20'),('spring','19'),('spring','18'),('spring','17'),
          ('summer','20'),('summer','19'),('summer','18'),('summer','17')]
df = pd.read_csv(mun_csv)
mun_name = df.loc[list(df[df['muncode'] == muncode].index)[0],'NOM_MUN'] ### Figures out the municipality name

### Loop over crop cycles and years here, or just select one
box_analysis_info = []
for ciclo, yr in ciclos:
    box_df_srt_cy, box_df_end_cy = read_in_AOI_box_info(ciclo, yr, muncode, asset_type=asset)
    if 'bid' in box_df_srt_cy.columns:
        if 'bid' in box_df_end_cy.columns:   
            box_df_srt_cy.loc[:,'bid'] = box_df_srt_cy['bid'].apply(lambda x: str(int(float(x))))
            box_df_end_cy.loc[:,'bid'] = box_df_end_cy['bid'].apply(lambda x: str(int(float(x))))
            bids = sorted([bid for bid in list(box_df_srt_cy['bid']) if bid in list(box_df_end_cy['bid'])])
            box_df_srt_cy = box_df_srt_cy[box_df_srt_cy['bid'].isin(bids)]
            box_df_end_cy = box_df_end_cy[box_df_end_cy['bid'].isin(bids)]
            box_df_srt_cy = box_df_srt_cy.sort_values('bid')
            box_df_end_cy = box_df_end_cy.sort_values('bid')
            xs = [x for x in  list(box_df_srt_cy['x'])]
            ys = [y for y in  list(box_df_srt_cy['y'])]
            image_ids_start = [ids for ids in list(box_df_srt_cy['image_id'])]
            image_ids_end = [ids for ids in list(box_df_end_cy['image_id'])]
            box_analysis_info.extend(list(zip(bids,xs,ys,image_ids_start,image_ids_end,[ciclo]*len(bids),[yr]*len(bids))))



In [6]:
start_time = time.time()        
mgr = mp.Manager()
pool_size = mp.cpu_count()
split_bids = np.array_split(box_analysis_info, pool_size-2)
pool = mp.Pool(processes=pool_size-2)
box_analysis_script=partial(box_analysis, muncode=muncode, mun_name=mun_name) 
pooled_img_hist_adc07, pooled_img_hist_adc16 = zip(*pool.map(box_analysis_script, split_bids))
pool.close()
pool.join()
pooled_img_hist_adc07 = total_img_hist(list(itertools.chain(*pooled_img_hist_adc07)))
pooled_img_hist_adc16 = total_img_hist(list(itertools.chain(*pooled_img_hist_adc16)))
### Sum up ADC level histograms to municipality level
pooled_mun_hist_07 = total_img_hist(pooled_img_hist_adc07, adc_level=False)        
pooled_mun_hist_16 = total_img_hist(pooled_img_hist_adc16, adc_level=False)
    
### Outputs -- This is where the histograms will be saved to
data_output_dir =  os.path.join(remote_sen_output_dir,'data',str(muncode).replace(str(muncode)[-3:],""),str(muncode))
data_adc07_dir  =  os.path.join(data_output_dir,'ADC07')
data_adc16_dir  =  os.path.join(data_output_dir,'ADC16')

for data_dir in [data_adc07_dir, data_adc16_dir]:#, data_adc07_chng_dir, data_adc16_chng_dir]:
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        
for total_hist in pooled_img_hist_adc07:
    adc = total_hist[1]
    cicloyr = total_hist[2]
    np.save(os.path.join(data_adc07_dir,cicloyr+"_adc07_"+adc+".npy"), total_hist[0])
    
for total_hist in pooled_img_hist_adc16:
    adc = total_hist[1]
    cicloyr = total_hist[2]
    np.save(os.path.join(data_adc16_dir,cicloyr+"_adc16_"+adc+".npy"), total_hist[0])
    
    
for total_hist in pooled_mun_hist_07:
    mun = total_hist[1]
    cicloyr = total_hist[2]
    np.save(os.path.join(data_output_dir,cicloyr+"_mun07_"+mun+".npy"), total_hist[0])
    
for total_hist in pooled_mun_hist_16:
    mun = total_hist[1]
    cicloyr = total_hist[2]
    np.save(os.path.join(data_output_dir,cicloyr+"_mun16_"+mun+".npy"), total_hist[0])
    
end_time = time.time()
duration = (end_time - start_time) / 60
print(f'The box analysis took {duration:.3f} minutes to run.')

/home/j/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
/home/j/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:73: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/home/j/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/home/j/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making th

/home/j/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:279: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/home/j/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:279: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/home/j/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:279: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order ch

/home/j/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:279: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/home/j/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:279: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/home/j/.local/lib/python3.7/site-packages/pyproj/crs/crs.py:279: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order ch

The box analysis took 5.123 minutes to run.
